In [1]:
from __future__ import print_function

import argparse
import pickle

import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms, datasets

In [197]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))
                                ])
trainset_labeled_import = pickle.load(open("../data/kaggle/train_labeled.p", "rb"))
trainset_unlabeled_import = pickle.load(open("../data/kaggle/train_unlabeled.p", "rb"))
validset_import = pickle.load(open("../data/kaggle/validation.p", "rb"))
train_labeled_loader = torch.utils.data.DataLoader(trainset_labeled_import, batch_size=32, shuffle=True)
train_unlabeled_loader = torch.utils.data.DataLoader(trainset_unlabeled_import, batch_size=256,
                                                     shuffle=True)

train_unlabeled_loader.dataset.train_labels = [-1 for i in range(len(train_unlabeled_loader.dataset.train_data))]
valid_loader = torch.utils.data.DataLoader(validset_import, batch_size=32, shuffle=True)

In [217]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(0.3)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #print(x.size())
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)
model = Net()
a = torch.randn(32, 1, 28, 28)
model.forward(Variable(a))

Variable containing:
-2.4230 -2.0857 -2.4230 -2.4230 -2.0699 -2.4230 -2.3342 -2.2204 -2.4225 -2.2946
-2.3742 -2.1285 -2.3742 -2.3742 -2.1718 -2.2952 -2.3677 -2.3742 -2.3742 -2.2335
-2.3802 -2.0872 -2.3802 -2.3778 -2.2749 -2.1184 -2.3277 -2.3802 -2.3802 -2.3802
-2.3835 -2.0849 -2.3227 -2.3835 -2.2288 -2.2904 -2.3030 -2.3835 -2.3835 -2.3031
-2.3603 -2.1602 -2.3546 -2.3603 -2.1501 -2.3465 -2.3603 -2.3174 -2.3603 -2.2879
-2.4241 -1.9623 -2.3352 -2.4241 -2.3873 -2.2147 -2.2514 -2.2815 -2.4241 -2.4241
-2.4228 -2.1905 -2.1599 -2.4228 -1.9354 -2.4228 -2.3395 -2.4228 -2.4228 -2.4228
-2.3715 -2.3715 -2.3715 -2.1984 -1.9549 -2.3715 -2.3637 -2.3715 -2.3715 -2.3715
-2.3742 -2.2783 -2.3742 -2.3090 -1.9508 -2.3742 -2.3282 -2.3742 -2.3742 -2.3742
-2.4529 -2.0989 -2.3338 -2.4529 -2.2214 -2.1298 -2.3025 -2.4529 -2.4529 -2.2167
-2.3855 -2.0893 -2.3855 -2.3855 -2.2527 -2.3422 -2.3047 -2.2136 -2.3855 -2.3255
-2.4127 -2.3684 -2.4127 -2.4096 -2.0278 -2.4127 -1.9314 -2.4127 -2.4127 -2.3853
-2.3540 -2.1958 -2.

In [248]:
class NetBN(nn.Module):
    def __init__(self):
        super(NetBN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=5, stride=1, padding=1,
                               bias=False)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=1,
                               bias=False)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=1,
                               bias=False)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=0, padding=0, ceil_mode=True) # change

        self.conv1_bn = nn.BatchNorm2d(64)
        self.conv2_bn = nn.BatchNorm2d(128)
        self.conv3_bn = nn.BatchNorm2d(256)
        self.conv1_drop = nn.Dropout2d(0.2)
        self.conv2_drop = nn.Dropout2d(0.3)
        self.conv3_drop = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(32 * 8, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = self.conv1_drop(F.relu(F.max_pool2d(self.conv1_bn(self.conv1(x)), (2,2))))

        x = self.conv2_drop(F.relu(F.max_pool2d(self.conv2_bn(self.conv2(x)), (2,2))))

        x = self.conv3_drop(F.relu(F.max_pool2d(self.conv3_bn(self.conv3(x)), (2,2))))
        x = x.view(-1, 32 * 8)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)

model = NetBN()
a = torch.randn(32, 1, 28, 28)
model.forward(Variable(a))


Variable containing:
-2.3885 -2.3885 -2.3885 -2.3885 -2.2074 -2.1908 -2.3885 -2.3885 -2.3885 -1.9966
-2.3157 -2.3157 -2.3013 -2.3157 -2.3157 -2.3157 -2.3157 -2.3157 -2.3157 -2.2044
-2.3847 -2.3847 -2.3847 -2.3847 -2.3421 -2.3847 -2.3847 -2.0694 -2.2659 -2.1107
-2.4014 -2.3823 -2.4014 -2.4014 -2.3507 -2.1476 -2.0121 -2.4014 -2.3287 -2.2820
-2.3900 -2.3900 -2.3900 -2.2110 -2.3900 -2.2174 -2.3900 -2.3900 -1.9653 -2.3900
-2.4278 -2.4278 -2.3567 -2.4278 -2.4278 -2.0843 -2.3407 -2.4278 -2.2954 -1.9469
-2.3720 -2.3720 -2.3720 -2.3720 -2.3720 -2.2710 -2.2748 -2.0849 -2.3720 -2.2082
-2.1829 -2.3394 -2.3394 -2.3394 -2.3076 -2.3394 -2.2784 -2.3394 -2.2605 -2.3115
-2.3772 -2.3772 -2.3772 -2.2878 -2.3772 -2.1622 -2.3428 -2.2212 -2.3772 -2.1643
-2.1357 -2.4849 -2.4849 -2.0544 -2.4335 -2.4849 -2.2952 -2.2487 -2.1664 -2.3530
-2.4213 -2.4213 -2.4213 -2.4213 -2.0300 -2.4213 -2.4165 -2.4213 -1.9950 -2.2021
-2.3539 -2.3599 -2.3599 -2.3599 -2.0902 -2.1211 -2.3599 -2.3599 -2.3556 -2.3599
-2.4513 -2.4910 -2.

In [249]:
import pickle
model = pickle.load(open("model.p", "rb"))
optimizer = optim.SGD(model.parameters(), lr=1.5, momentum=.8)

In [ ]:


my_criterion = MyCriterion()
x = Variable(torch.randn(11, 10).type(torch.FloatTensor))
y = Variable(torch.range(1,6, .5).type(torch.LongTensor))

a = torch.from_numpy(np.array([0]))
b = torch.from_numpy(np.array([1]))
c = torch.from_numpy(np.array([10.0]))

print(x)
# print(torch.from_numpy(np.array([10])))
my_criterion.train()

first_loss  = my_criterion(x, y, Variable(c.float()),  Variable(a))

second_loss = my_criterion(x, y, Variable(c.float()),  Variable(b))
print(first_loss, second_loss)

In [259]:
def train_labeled(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_labeled_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Labeled Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_labeled_loader.dataset),
                       100. * batch_idx / len(train_labeled_loader), loss.data[0]))



In [257]:
def train_unlabeled(epoch):
    model.train()
    for batch_idx, (data,_) in enumerate(train_unlabeled_loader):
        data = Variable(data)
        optimizer.zero_grad()
        output = model(data)
        target = Variable(torch.LongTensor(output.data.max(1)[1].numpy().reshape(-1)))
#         loss = my_criterion.forward(output, target, Variable(torch.LongTensor(epoch)), Variable(torch.LongTensor(2)))
#         my_criterion.backward(loss)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Unlabeled Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_unlabeled_loader.dataset),
                       100. * batch_idx / len(train_unlabeled_loader), loss.data[0]))

In [251]:
def validate(epoch, valid_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:

        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target)
        
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss.data.numpy()[0], correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [252]:
for epoch in range(1, 10):
    train(epoch)
    validate(epoch, valid_loader)

Train Labeled Epoch: 1 [0/3000 (0%)]	Loss: 2.250507
Train Labeled Epoch: 1 [320/3000 (11%)]	Loss: 2.306233
Train Labeled Epoch: 1 [640/3000 (21%)]	Loss: 2.251914
Train Labeled Epoch: 1 [960/3000 (32%)]	Loss: 2.073837
Train Labeled Epoch: 1 [1280/3000 (43%)]	Loss: 1.868223
Train Labeled Epoch: 1 [1600/3000 (53%)]	Loss: 1.891294
Train Labeled Epoch: 1 [1920/3000 (64%)]	Loss: 1.425283
Train Labeled Epoch: 1 [2240/3000 (74%)]	Loss: 1.460097
Train Labeled Epoch: 1 [2560/3000 (85%)]	Loss: 1.130306
Train Labeled Epoch: 1 [2880/3000 (96%)]	Loss: 0.732923

Test set: Average loss: 0.5256, Accuracy: 8822/10000 (88%)

Train Labeled Epoch: 2 [0/3000 (0%)]	Loss: 0.757906
Train Labeled Epoch: 2 [320/3000 (11%)]	Loss: 0.556363
Train Labeled Epoch: 2 [640/3000 (21%)]	Loss: 0.704225
Train Labeled Epoch: 2 [960/3000 (32%)]	Loss: 0.554556
Train Labeled Epoch: 2 [1280/3000 (43%)]	Loss: 0.683406
Train Labeled Epoch: 2 [1600/3000 (53%)]	Loss: 0.478517
Train Labeled Epoch: 2 [1920/3000 (64%)]	Loss: 0.380870
T

In [256]:
import pickle
# pickle.dump(model, open("model.p", "wb"))

In [260]:
model = pickle.load(open("model.p", "rb"))


In [261]:
for epoch in range(1, 10):
    train_labeled(epoch)
    train_unlabeled(epoch)

    validate(epoch, valid_loader)

Train Labeled Epoch: 1 [0/3000 (0%)]	Loss: 0.304256
Train Labeled Epoch: 1 [320/3000 (11%)]	Loss: 0.153869
Train Labeled Epoch: 1 [640/3000 (21%)]	Loss: 0.072204
Train Labeled Epoch: 1 [960/3000 (32%)]	Loss: 0.153059
Train Labeled Epoch: 1 [1280/3000 (43%)]	Loss: 0.019983
Train Labeled Epoch: 1 [1600/3000 (53%)]	Loss: 0.409640
Train Labeled Epoch: 1 [1920/3000 (64%)]	Loss: 0.058668
Train Labeled Epoch: 1 [2240/3000 (74%)]	Loss: 0.048152
Train Labeled Epoch: 1 [2560/3000 (85%)]	Loss: 0.191246
Train Labeled Epoch: 1 [2880/3000 (96%)]	Loss: 0.031356
Train Unlabeled Epoch: 1 [0/47000 (0%)]	Loss: 0.081837
Train Unlabeled Epoch: 1 [2560/47000 (5%)]	Loss: 0.078756
Train Unlabeled Epoch: 1 [5120/47000 (11%)]	Loss: 0.079416
Train Unlabeled Epoch: 1 [7680/47000 (16%)]	Loss: 0.065487
Train Unlabeled Epoch: 1 [10240/47000 (22%)]	Loss: 0.082107
Train Unlabeled Epoch: 1 [12800/47000 (27%)]	Loss: 0.062613
Train Unlabeled Epoch: 1 [15360/47000 (33%)]	Loss: 0.069898
Train Unlabeled Epoch: 1 [17920/4700

In [263]:
testset = pickle.load(open("../data/kaggle/test.p", "rb"))

test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [264]:
label_predict = np.array([])
model.eval()
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    temp = output.data.max(1)[1].numpy().reshape(-1)
    
    label_predict = np.concatenate((label_predict, temp))

In [265]:
label_predict

predict_label = pd.DataFrame(label_predict, columns=['label'], dtype=int)
predict_label.reset_index(inplace=True)
predict_label.rename(columns={'index': 'ID'}, inplace=True)

predict_label.head()

predict_label.to_csv('../data/kaggle/sample_submission.csv', index=False)